In [1]:
import argparse
from easydict import EasyDict as edict

parser = argparse.ArgumentParser()
parser.add_argument("--model", type=str, choices=["DiT-XL/2"], default="DiT-XL/2")
parser.add_argument("--vae", type=str, choices=["ema", "mse"], default="mse")
parser.add_argument("--image-size", type=int, choices=[256, 512], default=256)
parser.add_argument("--num-classes", type=int, default=1000)
parser.add_argument("--cfg-scale", type=float, default=4.0)
parser.add_argument("--num-sampling-steps", type=int, default=1000)
parser.add_argument("--seed", type=int, default=0)
parser.add_argument("--ckpt", type=str, default=None)

args = parser.parse_args([])  # Notebook에서는 [] 필수
args = edict(vars(args))


In [2]:


# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

"""
Sample new images from a pre-trained DiT.
"""
import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
from torchvision.utils import save_image
from diffusion import create_diffusion
from diffusers.models import AutoencoderKL
from download import find_model
from models import DiT_models
import argparse


# Setup PyTorch:
torch.manual_seed(args.seed)
torch.set_grad_enabled(False)
device = "cuda" if torch.cuda.is_available() else "cpu"

if args.ckpt is None:
    assert args.model == "DiT-XL/2", "Only DiT-XL/2 models are available for auto-download."
    assert args.image_size in [256, 512]
    assert args.num_classes == 1000

# Load model:
latent_size = args.image_size // 8
model = DiT_models[args.model](
    input_size=latent_size,
    num_classes=args.num_classes
).to(device)
# Auto-download a pre-trained model or load a custom DiT checkpoint from train.py:
ckpt_path = args.ckpt or f"DiT-XL-2-{args.image_size}x{args.image_size}.pt"
state_dict = find_model(ckpt_path)
model.load_state_dict(state_dict)
model.eval()  # important!
diffusion = create_diffusion(str(args.num_sampling_steps))
vae = AutoencoderKL.from_pretrained(f"stabilityai/sd-vae-ft-{args.vae}").to(device)



/home/scpark/miniconda3/envs/dual/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from solvers.common import NoiseScheduleVP, model_wrapper
from solvers.taylor.solver.gdual_solver import GDual_Solver

noise_schedule = NoiseScheduleVP(schedule="discrete", betas=torch.tensor(diffusion.betas))
solver = GDual_Solver(noise_schedule, 5, skip_type='time_uniform').to(device)
pt_file = '/home/scpark/logpx_samplers/logs/ablations/multi3/s5/step_00020000.pt'
state_dict = torch.load(pt_file, map_location='cpu', weights_only=False)['solver_state_dict']
solver.load_state_dict(state_dict, strict=True)


<All keys matched successfully>

In [13]:
# Labels to condition the model with (feel free to change):
class_labels = [207, 360, 387, 974, 88, 979, 417, 279]


# Create sampling noise:
n = len(class_labels)
z = torch.randn(n, 4, latent_size, latent_size, device=device)
y = torch.tensor(class_labels, device=device)
class_null = torch.tensor([1000] * len(y), device=device)

def inner_model_fn(x, t, cond, **kwargs):        
    return model.forward(x, t, cond)[:, :4]
        
model_fn = model_wrapper(
        inner_model_fn,
        noise_schedule,
        model_type="noise",
        guidance_type="classifier-free",
        condition=y,
        unconditional_condition=class_null,
        guidance_scale=1.5,
)

# Sample images:
samples = solver.sample(z, model_fn)['samples']
samples = vae.decode(samples / 0.18215).sample
save_image(samples, "sample.png", nrow=4, normalize=True, value_range=(-1, 1))

In [10]:
samples.shape

torch.Size([8, 3, 256, 256])